In [1]:
""""
The goal of this homework is to understand the approaches for taking derivatives and I will provide derivatives to the optimizer. I will use the truss problem with code available here: https://github.com/mdobook/resources/tree/main/exercises/tenbartruss

I will use the following methods:
    - Forward Finite Difference
    - Complex-Step Derivative
    - Algorithmic Differenctiation
"""

import numpy as np
from truss import truss

In [2]:
# Foward Finite Difference Derivative
"""
Foward finite-difference Gradient Compuation of a Vector-Valued Function
    inputs:
        - f: vector of functions of interest
        - x: point about which to compute the gradients
        - h: step size to use when calculating the gradient, defaults to 1e-6
    outputs:
        - J: jacobian of f wrt x
"""

def ff_dm_dA(f, x, h=1e-6):
    f0, _ = f(x)
    f_forward, _ = f(x + h)
    return (f_forward - f0) / h

def ff_dsigma_dA(f, x, h=1e-6):
    _, f0 = f(x)
    
    _, stress_h = truss(x+h)
    _, stress = truss(x)

    return (mass_h - mass) / h

In [5]:
def test_func(x):
    return np.exp(x) / np.sqrt(np.sin(x)**3 + np.cos(x)**3)

x = np.ones((10,))

ff_dm_dA(test_func, x, h=1e-6)


array([2.2778983, 2.2778983, 2.2778983, 2.2778983, 2.2778983, 2.2778983,
       2.2778983, 2.2778983, 2.2778983, 2.2778983])

In [ ]:
# Derivative of mass wrt cross-sectional areas (design variables) dm/dA_i for i = 1,...,n_x


# Derivative of stress (constraints) wrt to cross-sectional areas (design variables) dsigma_i/dA_j, for i=1,...,n_g and j=1,...,n_x. This will be a (n_g, n_x) shaped matrix where in this case n_g=n_x
